In [1]:
from qutip_qip.circuit import QubitCircuit
from qutip_qip.qir import circuit_to_qir

In [2]:
#Playing the CHSH game
circuit = QubitCircuit(3, num_cbits=2)
msg, here, there = range(3)

# Encode message
circuit.add_gate("RZ", targets=[msg], arg_value=0.123)

# Entangle qubits
circuit.add_gate("SNOT", targets=[here])
circuit.add_gate("CNOT", targets=[there], controls=[here])
circuit.add_gate("CNOT", targets=[here], controls=[msg])
circuit.add_gate("SNOT", targets=[msg])

# Players measure their qubits
circuit.add_measurement("Z", targets=[msg], classical_store=0)
circuit.add_measurement("Z", targets=[here], classical_store=1)

# Post process based on measurements
circuit.add_gate("X", targets=[there], classical_controls=[0])
circuit.add_gate("Z", targets=[there], classical_controls=[1])

with open('qutip_program.bc', 'wb') as file:
    file.write(circuit_to_qir(circuit, "bitcode"))
    
with open('qutip_program.ll', 'w') as file:
    file.write(circuit_to_qir(circuit, "text"))

Great! we now have a way of lowering high-level user code to QIR, but what does that look like?

In [3]:
print(circuit_to_qir(circuit, "text")[0:800])

; ModuleID = 'qutip_circuit'
source_filename = "qutip_circuit"

%Qubit = type opaque
%Result = type opaque

define void @main() #0 {
entry:
  call void @__quantum__qis__rz__body(double 1.230000e-01, %Qubit* null)
  call void @__quantum__qis__h__body(%Qubit* inttoptr (i64 1 to %Qubit*))
  call void @__quantum__qis__cnot__body(%Qubit* inttoptr (i64 1 to %Qubit*), %Qubit* inttoptr (i64 2 to %Qubit*))
  call void @__quantum__qis__cnot__body(%Qubit* null, %Qubit* inttoptr (i64 1 to %Qubit*))
  call void @__quantum__qis__h__body(%Qubit* null)
  call void @__quantum__qis__mz__body(%Qubit* null, %Result* null)
  call void @__quantum__qis__mz__body(%Qubit* inttoptr (i64 1 to %Qubit*), %Result* inttoptr (i64 1 to %Result*))
  %0 = call i1 @__quantum__qis__read_result__body(%Result* null)
  br i1 %0,


This isn't the nicest way to debug things (though really it should be only the compiler looking). Let's look at a control flow graph:

In [4]:
!opt -dot-cfg qutip_program.ll 

This is inadvisable as it may cause display problems. If
you REALLY want to taste LLVM bitcode first-hand, you
can force output with the `-f' option.

Writing '.main.dot'...


In [5]:
!dot -Tpng .main.dot -oqutip_program.png

![flow-diagram](qutip_program.png)

In [6]:
! qat --apply --profile default -S ./qutip_program.ll

/bin/bash: line 1: qat: command not found


In [11]:
import os

from numpy import size
os.stat("qutip_program.ll").st_size

2083